<a href="https://colab.research.google.com/github/TayJen/H_Tatarsan2022/blob/main/H_Tatarstan2022_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>